In [9]:
import xlwings as xw
import time
import datetime
import os 
import sqlite3
from copy import deepcopy

In [10]:
# STOLEN FROM https://www.geeksforgeeks.org/print-longest-common-substring/
# BUT MODIFIED
 
def subseq_detect(X, Y):
 
    m = len(X)
    n = len(Y)

    LCSuff = [[0 for i in range(n + 1)]
                 for j in range(m + 1)]
 
    length = 0
 
    row, col = 0, 0
 
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                LCSuff[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                LCSuff[i][j] = LCSuff[i - 1][j - 1] + 1
                if length < LCSuff[i][j]:
                    length = LCSuff[i][j]
                    row = i
                    col = j
            else:
                LCSuff[i][j] = 0

    if length == 0:
        return 0
 
    resultStr = ['0'] * length
 
    while LCSuff[row][col] != 0:
        length -= 1
        resultStr[length] = X[row - 1] # or Y[col-1]

        row -= 1
        col -= 1
 
    return len(resultStr)



def determine_insertion(old, new):

#         print(f"old: {old}")
#         print(f"new: {new}")
    common_len = subseq_detect(new, old)

    if common_len==0:
        insert_these = new

    elif common_len==len(old) and common_len==len(new):
        insert_these=[]

    elif old[:common_len]==new[len(new)-common_len:]:
        insert_these = new[:len(new)-common_len]

    else: 
        return determine_insertion(old[:-1],new)

    just_inserted = insert_these+old
#     print(f"insert_these: {insert_these}")
    return insert_these


def f_time_of_sales(inst_id, i):
    return f'=RTD("tt.rtd",,"TS","{inst_id}",{NUM_ROWS},CELL("address",A{i}))'

def f_price(inst_id, i):
    return f'=RTD("tt.rtd",,"TS","{inst_id}",{NUM_ROWS},CELL("address",B{i}))'

def f_quantity(inst_id, i):
    return f'=RTD("tt.rtd",,"TS","{inst_id}",{NUM_ROWS},CELL("address",C{i}))'

def f_side(inst_id, i):
    return f'=RTD("tt.rtd",,"TS","{inst_id}",{NUM_ROWS},CELL("address",D{i}))'

In [11]:


class xl_tt():
    def __init__(self, exchange_name, num_rows):
        self.exchange_name=exchange_name
        
        self.NUM_ROWS = num_rows
        self.group_names = []
        self.group_and_inst_names = {}
        self.wb = xw.Book()
        
        # inst_id sheet params
        self.wb.sheets.add('inst_ids')
        self.id_sheet=self.wb.sheets['inst_ids']
        self.id_horizontal_index = 1
        self.id_ranges_dict = {}
        self.id_dict_group = {}
        self.id_dict_all = {}
        
        # tns sheet params
        self.tns_ranges_dict_group = {}
        self.tns_ranges_dict_all = {}
    
    def xl_setup(self, group_name_inst_ids_list,id_setup=True):
        if id_setup is True:
            print('*** SETTING UP GROUP ID SHEET')
            self.id_setup(group_name_inst_ids_list)
        print('*** SETTING UP GROUP TNS SHEETS')
        self.tns_setup(group_name_inst_ids_list)
        
        
    def id_setup(self, group_name_inst_ids_list):
        for g in group_name_inst_ids_list:
            group_name=g[0]
            inst_names=g[1]
            self.set_group_inst_ids(inst_names,group_name)
        
        time.sleep(2)
        for g in group_name_inst_ids_list:
            group_name=g[0]
            result = self.record_group_inst_ids(group_name)
            for k,v in result.items():
                try: 
                    vint=int(v)
                except: 
                    print(f"***** PROBLEM WITH THE FOLLOWING:")
                    print(f"GROUP:{group_name}, INSTRUMENT:{k}, INSTRUMENT ID:{v}")
            
            
    def tns_setup(self, group_name_inst_ids_list):
        for g in group_name_inst_ids_list:
            group_name=g[0]
            inst_names=g[1]
            print(f"**** SETTING UP GROUP TNS SHEET FOR {group_name}")
            self.set_group_tns(inst_names,group_name)
            self.group_and_inst_names[group_name]=inst_names
            
            

    def set_one_tns(self, sheet, inst_name, horizontal_index):
        inst_id = self.get_inst_id(inst_name)
        
        if inst_id == '"Failed to find instrument"' or inst_id == 'Initializing':        
            sheet.range((1,horizontal_index)).value = inst_name
            sheet.range((2,horizontal_index)).value = inst_id            
            print(f" ** instrument_id for {inst_name} is fucked")
            return 


        sheet.range((1,horizontal_index)).value = inst_name
        sheet.range((2,horizontal_index)).value = inst_id          
        
        sheet.range((3,horizontal_index),(3,horizontal_index+4)).value = ['contract','time','side','price','quantity']
        
        for i in range(self.NUM_ROWS):
#             print('here')
            row = [inst_name,f_time_of_sales(inst_id, i+1),f_side(inst_id, i+1),f_price(inst_id, i+1),f_quantity(inst_id, i+1)]
            
            sheet.range((4+i,horizontal_index),(4+i,horizontal_index+4)).value=row

        print(f" ** TNS for instrument {inst_name} set successfully.")

        
    def set_group_tns(self, inst_names, group_name):
               
        self.group_names.append(group_name)
        
        try: 
            self.wb.sheets.add(group_name)
        except:
            self.wb.sheets[group_name].clear()
            
        sheet=self.wb.sheets[group_name]

        horizontal_index = 1
        
        self.tns_ranges_dict_group[group_name]={}
        self.tns_ranges_dict_all
        
        for inst_name in inst_names: 
            self.set_one_tns(sheet, inst_name, horizontal_index)
            self.tns_ranges_dict_all[inst_name]=''
            self.tns_ranges_dict_group[group_name][inst_name]=[(4,horizontal_index),(3+self.NUM_ROWS, horizontal_index+4)]
            
            horizontal_index+=6
#             print('here')
        
    def set_group_inst_ids(self, inst_names, group_name):

        self.id_sheet.range((1,self.id_horizontal_index)).value = group_name

        f_inst_ids=[f'=RTD("tt.rtd",,"Inst","{self.exchange_name}","{inst_name}")' for inst_name in inst_names]
        inst_ids = []
        i=2
        for f_inst_id in f_inst_ids:
            self.id_sheet.range((i,self.id_horizontal_index+1)).value = f_inst_id
            i+=1

        i=2
        for inst_name in inst_names:
            self.id_sheet.range((i,self.id_horizontal_index)).value = inst_name
            i+=1

        i=2
        for f_inst_id in f_inst_ids:
            inst_ids.append(self.id_sheet.range((i,self.id_horizontal_index+1)).value)
            i+=1
        
        if 'Not_Connected' in inst_ids:
            print(" NOT CONNECTED!")
        

        if '"Failed to find instrument"' in inst_ids:
            print("*********  Failed to find instrument id for at least one instrument  **********")
        
        
        if 'Initializing' in inst_ids:
            print("***************** instrument id initializing for at least one instrument *****************")            
        
        
        self.id_ranges_dict[group_name]= [(2,self.id_horizontal_index),(1+len(inst_names), self.id_horizontal_index+1)]

        self.id_horizontal_index += 3
        
        return inst_ids
    
    def record_group_inst_ids(self,group_name):
        range_tuples = self.id_ranges_dict[group_name]
        data = self.id_sheet.range(range_tuples[0],range_tuples[1]).value
        self.id_dict_group[group_name]={}
        for d in data: 
            inst_id = d[1]
            
            if inst_id == '"Failed to find instrument"':
                print(f"********* GROUP NAME: {group_name} - INST NAME: {d[0]} **********")
                print("*********  Failed to find instrument **********")
                print("********* for at least one instrument *********")


            if inst_id == 'Initializing':
                print(f"********* GROUP NAME: {group_name} - INST NAME: {d[0]} **********")
                print("***************** Initializing *****************")
                print("********* for at least one instrument **********")                
            
            self.id_dict_group[group_name][d[0]] = inst_id
            
            self.id_dict_all[d[0]]=inst_id
            
        return self.id_dict_group[group_name]
    
    def get_inst_id(self, inst_name):
        return self.id_dict_all[inst_name]
    
    def pull_tns(self, inst_name, group_name):
        ranges=self.tns_ranges_dict_group[group_name][inst_name]
        new_import=self.wb.sheets[group_name].range(ranges[0],ranges[1]).value

        # filter out rows that are empty or 'initializing'
        new_import=list(filter(lambda x: x[1]!='' and x[1]!='Initializing' , new_import))
        return new_import        
    
    
#     def determine_insertion(self, old, new):

# #         print(f"old: {old}")
# #         print(f"new: {new}")
#         common_len = subseq_detect(new, old)

#         if common_len==0:
#             insert_these = new

#         elif common_len==len(old) and common_len==len(new):
#             insert_these=[]

#         elif old[:common_len]==new[len(new)-common_len:]:
#             insert_these = new[:len(new)-common_len]

#         else: 
#             return determine_insertion(old[:-1],new)

#         print(f"insert_these: {insert_these}")
#         return insert_these    
    
#     def scan(self,):
#         for group in self.group_names:
#             for inst_name in self.group_and_inst_names[group]:
                

In [12]:
class xl_db():
    
    def __init__(self, DB_DIRECTORY, DB_NAME):
        
        if not os.path.exists(DB_DIRECTORY):
            print(f"CREATING DIRECTORY: {DB_DIRECTORY}")
            os.makedirs(DB_DIRECTORY)

        self.con = sqlite3.connect(DB_DIRECTORY+DB_NAME)
        self.cur = self.con.cursor()
        self.table_names = []

    def create_table(self, table_name):
        self.table_names.append(table_name)


        create_table = f'''CREATE TABLE IF NOT EXISTS {table_name} 
                           (date text, symbol text, time text, side text, price text, quantity text, insert_timestamp text)'''
        self.cur.execute(create_table)
        print(f"'{table_name}' -- TABLE CREATED.")

    def insert_multiple(self, table_name, insert_list):
        insert_list=deepcopy(insert_list)
        date = datetime.datetime.now().strftime("%Y-%m-%d")
        inserted_at = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

        # ADD transaction date + insert_stamp
        for i in range(len(insert_list)):
            insert_list[i].insert(0,date)
            insert_list[i].append(inserted_at)

        # INSERT AND COMMIT
        self.cur.executemany(f'INSERT INTO {table_name} VALUES (?,?,?,?,?,?,?)', insert_list)
        self.con.commit()
        print(f"{table_name} - {len(insert_list)} entries inserted at {inserted_at}")    
    
    

    
    

In [13]:
NUM_ROWS=10
EXCHANGE_NAME = 'MX'
DB_DIRECTORY = 'C:\\Users\\cyrus.maz\\Documents\\PY\\XL_TT_DB\\'
DB_NAME = 'MX_DB.db'
insts = [['outrights',['CGBM1','BAXM1']]]




In [14]:
NUM_ROWS=100
EXCHANGE_NAME = 'B3'
DB_DIRECTORY = 'C:\\Users\\cyrus.maz\\Documents\\PY\\XL_TT_DB\\'
DB_NAME = 'DI_1.db'
insts = [['jans',['DI1F22','DI1F23','DI1F24','DI1F25','DI1F26','DI1F27']]]




In [15]:
# INITIALIZE EXCEL SHEET:
xl=xl_tt(exchange_name=EXCHANGE_NAME, num_rows=NUM_ROWS)
xl.xl_setup(insts,id_setup=True)

# INITIALIZE DATABASE:
db = xl_db(DB_DIRECTORY, DB_NAME)


## CREAT A TABLE FOR EACH INSTRUMENT AND INITIALIZE OLD_INSERT DICT:
table_names=[]
old_insert = {}

for k,v in xl.group_and_inst_names.items():
    for inst_name in v:
        table_names.append(inst_name)
        db.create_table(inst_name)
        old_insert[inst_name]=[]


*** SETTING UP GROUP ID SHEET
*** SETTING UP GROUP TNS SHEETS
**** SETTING UP GROUP TNS SHEET FOR jans
 ** TNS for instrument DI1F22 set successfully.
 ** TNS for instrument DI1F23 set successfully.
 ** TNS for instrument DI1F24 set successfully.
 ** TNS for instrument DI1F25 set successfully.
 ** TNS for instrument DI1F26 set successfully.
 ** TNS for instrument DI1F27 set successfully.
'DI1F22' -- TABLE CREATED.
'DI1F23' -- TABLE CREATED.
'DI1F24' -- TABLE CREATED.
'DI1F25' -- TABLE CREATED.
'DI1F26' -- TABLE CREATED.
'DI1F27' -- TABLE CREATED.


In [8]:
new_import = {}
at_error = 0
c_error = 0
at_error_threshold = 3
c_error_threshold = 3
while True:
    if at_error>at_error_threshold:
        print("TOO MANY ATTRIBUTE ERRORS")
        break
    if c_error>c_error_threshold:
        print("TOO MANY COM_ERRORS")
        break    
    ## IMPORT TNS FOR ALL INSTRUMENTS:
    for group_name,v in xl.group_and_inst_names.items():
        for inst_name in v:
            try:
                new_import[inst_name]=xl.pull_tns(inst_name=inst_name, group_name=group_name)
                at_error = 0
                c_error = 0 
            except AttributeError as e:
                at_error+=1
                print(e)
                print(f"att_error: {at_error}")
            except Exception as e:
                c_error+=1
                print(e)
                print(type(e))
                print(f"c_error: {c_error}") 
                
                
#             print(f'NEW IMPORT: {inst_name}')
#             print(new_import[inst_name])
   

    all_inserted=[]
    ## DETERMINE WHAT NEEDS TO BE INSERTED INTO DATABASE 
    for group_name,v in xl.group_and_inst_names.items():
        for inst_name in v:
            insert_these = determine_insertion(old=old_insert[inst_name], new=new_import[inst_name])
#             print(f'INSERT THESE: {inst_name}')
#             print(insert_these)
#             print(f'OLD INSERT BEFORE: {inst_name}')
#             print(old_insert[inst_name])        
            if len(insert_these)>0:            
                all_inserted.append(insert_these)
                db.insert_multiple( table_name=inst_name, insert_list=insert_these)
                old=insert_these+old_insert[inst_name]
                old=old[:NUM_ROWS]
                old_insert[inst_name]=old
#                 print(f'OLD INSERT AFTER: {inst_name}')
#                 print(old)
        time.sleep(0.25)

KeyboardInterrupt: 

In [15]:
old

[['CGBM1', '12:33:02.320 ', 'SELL', '139.35', '1'],
 ['CGBM1', '12:33:02.185 ', 'SELL', '139.35', '1'],
 ['CGBM1', '12:32:49.624 ', 'BUY', '139.36', '1'],
 ['CGBM1', '12:32:49.624 ', 'BUY', '139.36', '1'],
 ['CGBM1', '12:32:49.604 ', 'BUY', '139.36', '4'],
 ['CGBM1', '12:32:49.590 ', 'BUY', '139.36', '1'],
 ['CGBM1', '12:32:49.590 ', 'BUY', '139.36', '1'],
 ['CGBM1', '12:32:49.590 ', 'BUY', '139.36', '1'],
 ['CGBM1', '12:32:49.590 ', 'BUY', '139.36', '1'],
 ['CGBM1', '12:32:49.578 ', 'SELL', '139.36', '1']]

In [ ]:
# group_name = 'jans'
# inst_name = 'DI1F24'

group_name = 'jans'
inst_name = 'DI1F22'

new_import = xl.pull_tns(inst_name=inst_name, group_name=group_name)

In [ ]:
vec=[10,9,8,7,6,5,4,3,2,1,]     
NUM_ROWS=4

N=len(vec)
# vec = list(range(N))
for old_start in range(N):
#     print(list(range(old_start+1,N-1)))
    
    for old_end in range(old_start+1,N):
#         print(f"  old_start: {old_start}, old_end:{old_end}")
        for new_start in range(old_start,old_end+1):
#             print(f"    new_start: {new_start}")
            for new_end in range(old_end,N+1):
                if new_start==new_end:
                    continue
                old = vec[N-old_end:N-old_start]
                new = vec[N-new_end:N-new_start]
                if len(new)<len(old):
                    continue
                print(f"old: {old}")
                print(f"new: {new}")
                determine_insertion(old,new,NUM_ROWS)
                print('\n\n')


In [41]:
import time
